# Jet Background subtraction via NN, Multiplicty and Area based methods

In [4]:
import ROOT
import os.path 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import datetime

from sklearn.inspection import permutation_importance
import matplotlib as plt

%load_ext tensorboard
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Config file defining ML varibles, training sets and data 

In [28]:
dataset = ['AuAu_200GeV_0to10cent_R2','AuAu_200GeV_0to10cent_R4','AuAu_200GeV_20to40cent_R2','AuAu_200GeV_20to40cent_R4']
AuAu_200GeV_0to10cent_R2 = [
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin0.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin1.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin2.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin3.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin4.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin5.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin6.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin7.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin8.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin9.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin10.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin11.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin12.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin13.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin14.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin15.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin16.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin17.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_0to10cent_ptbin18.root']

AuAu_200GeV_20to40cent_R2 = [
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin0.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin1.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin2.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin3.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin4.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin5.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin6.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin7.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin8.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin9.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin10.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin11.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin12.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin13.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin14.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin15.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin16.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin17.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R02_20to40cent_ptbin18.root']

AuAu_200GeV_0to10cent_R4 = [
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin0.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin1.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin2.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin3.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin4.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin5.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin6.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin7.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin8.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin9.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin10.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin11.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin12.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin13.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin14.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin15.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin16.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin17.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_0to10cent_ptbin18.root']

AuAu_200GeV_20to40cent_R4 = [
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin0.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin1.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin2.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin3.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin4.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin5.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin6.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin7.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin8.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin9.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin10.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin11.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin12.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin13.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin14.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin15.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin16.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin17.root',
    '/home/tmengel/jet-background-subtraction/data/jet-trees-formatted/200GevAuAu_R04_20to40cent_ptbin18.root']

jetTree = ['outTree']
MLvaribles = ['weight',
            'scaledweight',
            'median_pt_over_area','median_npart_over_area',
            'fastjet_rho',
            'event_average_pt',
            'AreaBased',
            'NumberBased',
            'event_median_pt',
            'jetpt','jeteta','jetphi','jetpt_pythia','jetpt_pythia_fraction','jetarea',
            'average_track_pt','median_track_pt','jetnparts','track0pt','track1pt','track2pt','track3pt','track4pt','jetangularity',
            'trackptvariance','trackptskewness','trackptkurtosis']

IgnoreVaribles = ['scaledweight','AreaBased', 'NumberBased']

InputVaribles = [ 'median_pt_over_area','median_npart_over_area',
            'event_average_pt',
            'AreaBased',
            'NumberBased',
            'event_median_pt',
            'jetpt','jeteta','jetphi','jetarea',
            'average_track_pt','median_track_pt','jetnparts','track0pt','track1pt','track2pt','track3pt','track4pt','jetangularity',
            'trackptvariance','trackptskewness','trackptkurtosis']


# InputVaribles = ['median_pt_over_area', 'jetpt', 'jetarea', 'average_track_pt', 'median_track_pt','jetnparts','track0pt','track1pt','track2pt', 'track3pt','track4pt', 'jetangularity', 'trackptvariance']
Target = ['jetpt_pythia'] # 'jetpt_pythia'

# Function definitions

In [32]:
def generate_model():
    """
    Keras model architecture for neural network from similiar to Rudy paper 
    """
    # get length of input varibles from config
    inputdim = len(InputVaribles)
    return keras.Sequential(
    [
        layers.Dense(100,input_shape=(inputdim,),activation='relu'),
        layers.Dense(100, activation='tanh'),
        layers.Dense(100, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(1, activation='relu')
    ])

def loaddata(filename):
    """
    Read data from ROOT files: x (input varibles), y (target), z (spectators)
    """
    # Read data from ROOT files
    TreeName = 'jetTree'
    data= ROOT.RDataFrame(TreeName, filename).AsNumpy()
    x = np.vstack([data[var] for var in InputVaribles]).T
    y = np.vstack([data[var] for var in Target]).T
    z = np.vstack([data[var] for var in IgnoreVaribles]).T
    return x, y, z  

def splitdata(filelistname, filelist, runname):
    """
    Split ROOT files into train/test/validaiton
    """

    ROOT.EnableImplicitMT() #enable mult-threading
    # define varibles in rootfile
    columns = ROOT.std.vector["string"](MLvaribles)
    chain = ROOT.TChain(jetTree[0])

    # create TChain of input filelist
    for filename in filelist:
        chain.Add(filename)

    # fstrings for split data paths
    TreeName = 'jetTree'
    testfile = f'/home/tmengel/jet-background-subtraction/train-test-split/{runname:s}_training_{filelistname:s}.root'
    trainfile = f'/home/tmengel/jet-background-subtraction/train-test-split/{runname:s}_testing_{filelistname:s}.root'
    valfile = f'/home/tmengel/jet-background-subtraction/train-test-split/{runname:s}_validation_{filelistname:s}.root'

    # split root files with rdataframe filters  
    formjets = ROOT.RDataFrame(chain) 

    formjets = formjets.Filter("jetpt_pythia > 5.0","pythia pt filtering")

    formjets.Filter("gRandom->Uniform(0, 10) < 8 ", "Select 90{%} of jets for training")\
        .Snapshot(TreeName, testfile, columns)
    formjets.Filter("gRandom->Uniform(0, 10) > 8 && gRandom->Uniform(0, 1) < 0.5 ", "Select 10{%} of jets for testing")\
        .Snapshot(TreeName, trainfile, columns)
    formjets.Filter("gRandom->Uniform(0, 10) > 8 && gRandom->Uniform(0, 1) > 0.5 ", "Select 10{%} of jets for validation")\
        .Snapshot(TreeName, valfile, columns)

    # formjets.Filter("gRandom->Uniform(0, 10)*jetpt > 150 && gRandom->Uniform(0, 1) < 0.8 ", "Select 80{%} high pt-biased jets for training")\
    #     .Snapshot(TreeName, trainfile, columns)
    # formjets.Filter("gRandom->Uniform(0, 10)*jetpt > 150 && gRandom->Uniform(0, 1) > 0.8 ", "Select 20{%} high pt-biased of jets for validation")\
    #     .Snapshot(TreeName, valfile, columns)
    # formjets.Filter("gRandom->Uniform(0, 10)*jetpt < 150 ", "Select low pt-biased jets for testing")\
    #     .Snapshot(TreeName, testfile, columns)


def trainModel(filelistname, runname):
    """
    Trains the neural network and ckpts using tensorboard
    """
    # fstrings for file paths
    
    validationfile = f'/home/tmengel/jet-background-subtraction/train-test-split/{runname:s}_validation_{filelistname:s}.root'
    trainingfile = f'/home/tmengel/jet-background-subtraction/train-test-split/{runname:s}_training_{filelistname:s}.root'
    checkpoint_path = f'/home/tmengel/jet-background-subtraction/checkpoints/{runname:s}_{filelistname:s}.ckpt'

    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,  verbose=1)


    log_dir = f'/home/tmengel/jet-background-subtraction/checkpoints/{runname:s}_{filelistname:s}_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # get data from rootfiles
    x,y,z = loaddata(trainingfile)
    x_val,y_val,z_val = loaddata(validationfile)
    
    # Set up model 
    model = generate_model()
    model.compile(loss='mse', optimizer='adam')
    model.summary()
    
    batch_size = 128
    epochnum = 20
    model.fit(x=x,y=y, epochs=epochnum,  batch_size=batch_size, validation_data=(x_val,y_val),callbacks=[tensorboard_callback,cp_callback])

def evaluateAllMethods(filelistname,runname):
    """
    Produces final result root file: ResultVaribles = ['NeuralNetworkBased','AreaBased','MultiplicityBased','pythiapt','rawjetpt','weights']
    """
    # fstrings for file paths
    testingfile = f'/home/tmengel/jet-background-subtraction/train-test-split/{runname:s}_testing_{filelistname:s}.root'
    checkpoint_path = f'/home/tmengel/jet-background-subtraction/checkpoints/{runname:s}_{filelistname:s}.ckpt'
    checkpoint_dir_paper = os.path.dirname(checkpoint_path)

    # Result root filename
    rootfilename = f'/home/tmengel/jet-background-subtraction/results/results_{runname:s}_{filelistname:s}.root'
    TreeName = 'resultsTree'

    # get testing jets for corresponding R value
    x_test,y_test,z_test = loaddata(testingfile)

    # load checkpoint
    model = generate_model()
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.load_weights(checkpoint_path)
   
    # get NN prediction

    NeuralNetworkBased = model(x_test)

    # Linear Models
    weights = z_test[:,0]
    AreaBased = z_test[:,1]
    MultiplicityBased = z_test[:,2]
    # truth value
    pythiapt = y_test[:,0]
    # other values for result file
    rawjetpt = x_test[:,1]
    

    # varibles to fill TTree 
    NNpred = np.empty((1), dtype="float32")
    ABpred = np.empty((1), dtype="float32")
    MBpred = np.empty((1), dtype="float32")
    truthPt = np.empty((1), dtype="float32")
    rawPt = np.empty((1), dtype="float32")
    Scaledweights = np.empty((1), dtype="float32")

    # TTree architechure
    root_file = ROOT.TFile(rootfilename, "RECREATE")
    tree = ROOT.TTree(TreeName, TreeName)
    tree.Branch("NeuralNetworkBased", NNpred, "NeuralNetworkBased/F")
    tree.Branch("AreaBased", ABpred, "AreaBased/F")
    tree.Branch("MultiplicityBased", MBpred, "MultiplicityBased/F")
    tree.Branch("pythiapt", truthPt, "pythiapt/F")
    tree.Branch("rawjetpt", rawPt, "rawjetpt/F")
    tree.Branch("weights", Scaledweights, "weights/F")

    # fill TTree
    for i in range(pythiapt.shape[0]):
        NNpred[0] = NeuralNetworkBased[i]
        ABpred[0] = AreaBased[i]
        MBpred[0] = MultiplicityBased[i]
        truthPt[0] = pythiapt[i]
        rawPt[0] = rawjetpt[i]
        Scaledweights[0] = weights[i] 
        tree.Fill()
        
    root_file.Write()  

def TheWholeShabang(filelistname, filelist, runname):
    """
    Complete process for Area/Multiplicity/NueralNetwork based subtraction
    """
    # split rootfiles into train/test/validate (80/10/10). output to splitdata/(train/test/validate)_{filelistname}.root
    splitdata(filelistname, filelist, runname)
    # train NN: ADAM, 20 epochs, mean_squared_error. output to /home/tmengel/jet-background-subtraction/checkpoints/{filelistname}.ckpt
    trainModel(filelistname,runname)
    # create results for trained NN applied to test file, area based and multiplicity based. output to results/results_{filelistname}.root
    evaluateAllMethods(filelistname,runname)


In [33]:
splitdata(dataset[0],AuAu_200GeV_0to10cent_R2,'uniform_split')

In [34]:
trainModel(dataset[0],'uniform_split')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 100)               2300      
                                                                 
 dense_25 (Dense)            (None, 100)               10100     
                                                                 
 dense_26 (Dense)            (None, 100)               10100     
                                                                 
 dense_27 (Dense)            (None, 50)                5050      
                                                                 
 dense_28 (Dense)            (None, 1)                 51        
                                                                 
Total params: 27,601
Trainable params: 27,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1431/1431 [============================

In [19]:

runname = 'uniform_split'
filelistname = dataset[0]
testingfile = '/home/tmengel/jet-background-subtraction/train-test-split/uniform_split_testing_AuAu_200GeV_0to10cent_R2.root'

checkpoint_path = f'/home/tmengel/jet-background-subtraction/checkpoints/{runname:s}_{filelistname:s}.ckpt'
checkpoint_dir_paper = os.path.dirname(checkpoint_path)

x_test,y_test,z_test = loaddata(testingfile)

model = generate_model()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=tf.keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None))
model.load_weights(checkpoint_path)
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 100)               1400      
                                                                 
 dense_17 (Dense)            (None, 100)               10100     
                                                                 
 dense_18 (Dense)            (None, 50)                5050      
                                                                 
 dense_19 (Dense)            (None, 1)                 51        
                                                                 
Total params: 16,601
Trainable params: 16,601
Non-trainable params: 0
_________________________________________________________________


In [21]:
scoring = ['r2', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error']
r = permutation_importance(model, x_test, y_test, n_repeats=30, random_state=0, scoring=scoring)

In [27]:
for metric in r:
    print(f'{metric}')
    r_temp = r[metric]
    for i in r_temp.importances_mean.argsort()[::-1]:
        print(f'    {InputVaribles[i]:<15}'
                f'    {r_temp.importances_mean[i]:.3f}'
                f' +/- {r_temp.importances_std[i]:.3f}')


r2
    jetpt              0.609 +/- 0.004
    track0pt           0.167 +/- 0.001
    track1pt           0.097 +/- 0.001
    track2pt           0.042 +/- 0.000
    trackptvariance    0.028 +/- 0.000
    track3pt           0.014 +/- 0.000
    track4pt           0.007 +/- 0.000
    jetnparts          0.006 +/- 0.000
    average_track_pt    0.001 +/- 0.000
    median_track_pt    0.001 +/- 0.000
    median_pt_over_area    0.000 +/- 0.000
    jetarea            0.000 +/- 0.000
    jetangularity      -0.000 +/- 0.000
neg_mean_absolute_percentage_error
    jetpt              0.391 +/- 0.002
    track0pt           0.217 +/- 0.001
    track1pt           0.166 +/- 0.001
    track2pt           0.082 +/- 0.001
    trackptvariance    0.062 +/- 0.001
    track3pt           0.027 +/- 0.000
    jetnparts          0.009 +/- 0.000
    track4pt           0.008 +/- 0.000
    average_track_pt    0.003 +/- 0.000
    median_track_pt    0.001 +/- 0.000
    median_pt_over_area    0.000 +/- 0.000
    jetarea    

In [ ]:
%tensorboard --logdir 